In [1]:
import sys
import json
import ctypes  # An included library with Python install.
from ipyleaflet import Marker, LayersControl, basemaps, GeoData, Icon, WidgetControl, Choropleth, Map, CircleMarker

from ipywidgets import HTML, HBox, IntSlider, Image, Dropdown, Label, Button, interact, jsdlink, Layout, Output, HBox, RadioButtons
from ipywidgets.embed import embed_data
import geopandas as gpd
import fiona
from shapely.geometry import shape
import glob
import folium
import ipywidgets as widgets
from branca.colormap import linear
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot
import pandas as pd
from datetime import datetime
import time
import matplotlib
import numpy as np
import math
import plotly.graph_objects as go




In [2]:
if __name__ == '__main__':
    
    
    #define map
    m = Map(center=(52.3,8.0), zoom = 13, basemap = basemaps.Stamen.Toner, layout=Layout(width='100%', height='600px'))
         
    display(m)
    
    

    
    # add river catchment
    river_catchment_style={"color": "green",  "weight":1, "fillColor": "white"}
    river_catchment_file = "GIS/RiverCatchment.shp"
    river_catchment = gpd.read_file(river_catchment_file)
    river_catchment_geo_shape = river_catchment.to_crs(4326)
    river_catchment_shape_to_map = GeoData(geo_dataframe = river_catchment_geo_shape, style = river_catchment_style)
    m.add_layer(river_catchment_shape_to_map) 
     
    #add SSC connectivity
    SSC_conn_file = "GIS/SSC_links2.shp"
    SSC_con = gpd.read_file(SSC_conn_file)
    geo_SSC_con_shape = SSC_con.to_crs(4326)
    geo_SSC_con_shape_to_map = GeoData(geo_dataframe = geo_SSC_con_shape, style = {"color": "green", "fillColor": "green", "weight":5})
    m.add_layer(geo_SSC_con_shape_to_map)
    
    #add river subcatchments
    subcatchments_style={"color": "black",  "weight":1, "fillColor": "white"}
    subcatchments_file = "GIS/sub-catchments.shp"
    subcatchments = gpd.read_file(subcatchments_file)
    subcatchments_geo_shape = subcatchments.to_crs(4326)
    subcatchments_shape_to_map = GeoData(geo_dataframe = subcatchments_geo_shape, style = subcatchments_style)
    m.add_layer(subcatchments_shape_to_map)
    
    
    #add SCCs 
    style={"color": "black",  "weight":1, "fillColor": "white"}
    hover_style={"color": "red",  "weight":2, "fillColor": "red"}
    
    html_SSC = HTML("Hover Over SSCs or River Reaches")
    html_SSC.layout.margin = "0px 20px 20px 20px"
    control = WidgetControl(widget=html_SSC, position="topright")
    m.add_control(control)
    
    def update_html_SSC_name(feature, **kwargs):
        html_SSC.value = """<h4>SSC: {}</h4>
          """.format(feature["properties"]["NAME"])
        show_SSC_sankey(feature["properties"]["NAME"])
    
    SSC_file = "GIS/SSCs/collated_SSCs.shp"
    SSCs_style={"color": "black",  "weight":1, "fillColor": "white"}
    SSCs = gpd.read_file(SSC_file)
    SSCs_geo_shape = SSCs.to_crs(4326)
    SSCs_hover_style={"color": "red",  "weight":2, "fillColor": "red"}
    SSCs_layer = GeoData(geo_dataframe = SSCs_geo_shape, style = SSCs_style, hover_style = SSCs_hover_style)
    SSCs_layer.on_hover(update_html_SSC_name)
    m.add_layer(SSCs_layer)
    lonCent = (SSCs_geo_shape.bounds.maxx + SSCs_geo_shape.bounds.minx).mean()/2
    latCent = (SSCs_geo_shape.bounds.maxy + SSCs_geo_shape.bounds.miny).mean()/2
    m.center = (latCent,lonCent)
    
    #add river shape file to map
    def update_html_river_reach_name(feature, **kwargs):
        html_SSC.value = """<h4>River Reach: {}</h4>
            """.format(feature["properties"]["name1"])
        show_CSO_sankey(feature["properties"]["name1"])
                                                     
    river_file = "GIS/RiverReaches.shp"
    river_shape = gpd.read_file(river_file)
    river_geo_shape = river_shape.to_crs(4326)
    river_style={"color": "blue",  "weight":2, "fillColor": "blue"}
    river_hover_style={"color": "red",  "weight":4, "fillColor": "red"}
    river_shape_to_map = GeoData(geo_dataframe = river_geo_shape, style = river_style, hover_style = river_hover_style)
    river_shape_to_map.on_hover(update_html_river_reach_name)
    m.add_layer(river_shape_to_map)
    
    river_file = "GIS/RiverReaches2.shp"
    river_shape = gpd.read_file(river_file)
    river_geo_shape = river_shape.to_crs(4326)
    river_style={"color": "blue",  "weight":2, "fillColor": "blue"}
    river_hover_style={"color": "red",  "weight":4, "fillColor": "red"}
    river_shape_to_map = GeoData(geo_dataframe = river_geo_shape, style = river_style, hover_style = river_hover_style)
    #river_shape_to_map.on_hover(update_html_river_reach_name)
    m.add_layer(river_shape_to_map)
    
    #add SCC links
    links_file = "GIS/links.shp"
    links_shape = gpd.read_file(links_file)
    links_geo_shape = links_shape.to_crs(4326)
    links_style={"color": "brown",  "weight":2, "fillColor": "white"}
    links_shape_to_map = GeoData(geo_dataframe = links_geo_shape, style = links_style)
    #m.add_layer(links_shape_to_map)
     
    #add STWs
    STW_point_file = "GIS/STW_locations_4326.shp"
    icon_url = "https://icons.iconarchive.com/icons/custom-icon-design/pretty-office-9/128/triangle-icon.png"
    STW_point_icon = Icon(icon_url=icon_url, icon_size=[20, 20])
    with fiona.open(STW_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            STW_name = properties["SimName"]
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = STW_point_icon, title = STW_name)
            message = HTML(value="%s"%(STW_name))
            mark.popup = message
            m.add_layer(mark)
    #add CSOs        
    CSO_point_file = "GIS/CSOs_4326.shp"
    #CSO_icon_url = "https://icons.iconarchive.com/icons/fatcow/farm-fresh/32/shape-square-icon.png"
    CSO_icon_url = "https://icons.iconarchive.com/icons/custom-icon-design/flatastic-9/256/Stop-red-icon.png"
    CSO_point_icon = Icon(icon_url=CSO_icon_url, icon_size=[15, 15])
    with fiona.open(CSO_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            ID = properties["node_id"]
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = CSO_point_icon, title = ID)
            message = HTML(value="%s"%(ID))
            mark.popup = message
            m.add_layer(mark)
            
    #add nodes        
    nodes_point_file = "GIS/nodes_4326.shp"
    nodes_icon_url = "https://icons.iconarchive.com/icons/sekkyumu/developpers/32/Orange-Ball-icon.png"
    nodes_point_icon = Icon(icon_url=nodes_icon_url, icon_size=[5, 5])
    with fiona.open(nodes_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = nodes_point_icon)
            #m.add_layer(mark)
            
    sankey_window = widgets.Output(layout = widgets.Layout(height='300px', width='500px'))
    CSO_window = widgets.Output(layout = widgets.Layout(height='300px', width='500px'))
    
    #add river reach points
    river_point_file = "GIS/River_points.shp"
    icon_url = "https://icons.iconarchive.com/icons/custom-icon-design/pretty-office-9/128/circle-icon.png"
    river_point_icon = Icon(icon_url=icon_url, icon_size=[20, 20])
    with fiona.open(river_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            reach_name = properties["NAME"]
            yx_coords = [geom.y, geom.x]
            #mark = Marker(location=yx_coords, icon = river_point_icon, title = reach_name)
            circle_marker = CircleMarker()
            circle_marker.location = (geom.y, geom.x)
            circle_marker.radius = 5
            circle_marker.color = "blue"
            circle_marker.fill_color = "blue"
            circle_marker.fill_opacity = 1
            m.add_layer(circle_marker)
            reach_name = reach_name.replace("_","")
    
    #Add SSC points
    #point_colours = {"SSC_BartonBridge_SC": "orange",
    #                 "SSC_CulverSt_SC": "green",
    #                 "SSC_WooleySt_SC": "purple",
    #                 "SSC_Church_St_SC": "black",
    #                 "SSC_SilverSt_SC": "yellow",
    #                 "SSC_BartonFarm_SC": "red"}
    SSC_points_file = "GIS/SSC_points.shp"
    with fiona.open(SSC_points_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            SSC_name = properties["SSC_name"]
            yx_coords = [geom.y, geom.x]
            #mark = Marker(location=yx_coords, icon = river_point_icon, title = reach_name)
            circle_marker = CircleMarker()
            circle_marker.location = (geom.y, geom.x)
            circle_marker.radius = 5
            circle_marker.color = "brown"#point_colours[SSC_name]
            circle_marker.fill_color = "brown"#point_colours[SSC_name]
            circle_marker.fill_opacity = 1
            m.add_layer(circle_marker)
    
    sankey_line = HBox([sankey_window,CSO_window])
    display(sankey_line)
 
    source = [0, 1, 2, 3, 4]
    target = [5, 5, 4, 0, 0]
    value = [1, 1, 1, 1, 1]
    label_nodes = ["SSC_BartonBridge_SC", "SSC_CulverSt_SC",
                   "SSC_WooleySt_SC", "SSC_Church_St_SC",
                   "SSC_SilverSt_SC", "SSC_BartonFarm_SC"]
    SSC_colours = ["orange", "green", "purple",
                    "black", "yellow", "red"]
    
    #display(sankey_window)
    link = dict(source = source, target = target, value = value)
    node = dict(label = label_nodes, pad=50, thickness=5, color = SSC_colours)
    data = go.Sankey(link = link, node=node)
    fig = go.Figure(data)
    fig.update_layout(
        hovermode = 'x',
        title="Links between SSCs",
        font=dict(size = 10, color = 'black'),
        paper_bgcolor="white",
        width=500, height=300)
    #fig.show()
    with sankey_window:
        sankey_window.clear_output()
        fig.show() 
        
    source_CSO = [2, 3, 4, 5, 6]
    target_CSO = [0, 0, 0, 1, 1]
    value_CSO = [1, 1, 1, 1, 1]
    label_nodes_CSO = ["Avon_6","Avon_5","SSC_WooleySt","SSC_SilverSt","SSC_ChurchSt","SSC_BartonBridge","SSC_CulverSt"]
    colours_CSO = ["orange", "green", "purple", "black", "yellow", "red"]
    #display(CSO_window)
    
    link_CSO = dict(source = source_CSO, target = target_CSO, value = value_CSO)
    node_CSO = dict(label = label_nodes_CSO, pad=50, thickness=5, color = colours_CSO)
    data_CSO = go.Sankey(link = link_CSO, node=node_CSO)
    fig_CSO = go.Figure(data_CSO)
    fig_CSO.update_layout(
        hovermode = 'x',
        title="Spills to river reaches",
        font=dict(size = 10, color = 'black'),
        paper_bgcolor="white",
        width=500, height=300)
    #fig.show()
    with CSO_window :
        CSO_window.clear_output()
        fig_CSO.show() 
    
    def show_SSC_sankey(focal_SCC):
        SSC_index = label_nodes.index(focal_SCC)
    
        source_abb = []
        target_abb = []
        for i in range(0, len(source)):
            if SSC_index == source[i]:
                source_abb.append(SSC_index) 
                target_abb.append(target[i])
        for i in range(0, len(target)):
            if SSC_index == target[i]:
                target_abb.append(SSC_index) 
                source_abb.append(source[i])
        link = dict(source = source_abb, target = target_abb, value = value)
        node = dict(label = label_nodes, pad=50, thickness=5, color = SSC_colours)
        data = go.Sankey(link = link, node=node)
        fig = go.Figure(data)
        fig.update_layout(
        hovermode = 'x',
        title="Links between SSCs",
        font=dict(size = 10, color = 'black'),
        paper_bgcolor="white",
        width=500, height=300)
        with sankey_window:
            sankey_window.clear_output()
            fig.show()
        #fig.show()
    def show_CSO_sankey(focal_reach):
        try:
            CSO_index = label_nodes_CSO.index(focal_reach)
        except:
            return
        source_abb = []
        target_abb = []
        for i in range(0, len(source_CSO)):
            if CSO_index == source_CSO[i]:
                source_abb.append(CSO_index) 
                target_abb.append(target_CSO[i])
        for i in range(0, len(target_CSO)):
            if CSO_index == target_CSO[i]:
                target_abb.append(CSO_index) 
                source_abb.append(source_CSO[i])
        link_CSO = dict(source = source_abb, target = target_abb, value = value_CSO)
        node_CSO = dict(label = label_nodes_CSO, pad=50, thickness=5, color = colours_CSO)
        data_CSO = go.Sankey(link = link_CSO, node=node_CSO)
        fig_CSO = go.Figure(data_CSO)
        fig_CSO.update_layout(
            hovermode = 'x',
            title="Spills to river reaches",
            font=dict(size = 10, color = 'black'),
            paper_bgcolor="white",
            width=500, height=300)
        with CSO_window:
            CSO_window.clear_output()
            fig_CSO.show()
    

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…